In [1]:
import os
from datetime import datetime

from dotenv import load_dotenv
import pymysql
import pymysql.cursors

load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_SCHEMA = os.getenv("DB_SCHEMA")

con = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, port=3306, database=DB_SCHEMA)

data = []
month = 8
monthName = "august"
year = 2024

with pymysql.cursors.DictCursor(con) as cursor:
    queryString = f"SELECT e.empId, et.employeeType, et.dailyWage, ah.month, ah.year, ah.days, ah.ot_hours FROM `employees` AS e\
        JOIN `employeetypes` AS et ON e.employeeTypeId = et.employeeTypeId\
        JOIN `attendance_history` AS ah ON e.empId = ah.emp_id\
        WHERE ah.month = {month} AND ah.year = {year}"
    cursor.execute(queryString)

    datas = cursor.fetchall()

    count = 0

    for data in datas:
        with con.cursor() as salaryCursor:
            # basic_amount = number of working days * dailywage
            basic_amount = data['days'] * data['dailyWage']
            # pf = basic_amount * 12 / 100
            pf = basic_amount * 12 / 100
            # esi = basic_amount * 0.75 / 100
            esi = basic_amount * 0.75 / 100
            # gross_amount = basic_amount - pf - esi
            gross_amount = basic_amount - pf - esi
            ### For good attendance
            # if working days > 25 : basic_amount + 5.6; else if > 24: basic_amount + 4.2; else if > 20: basic_amount + 2.8; else if > 16: basic_amount + 1.4
            good_attendance = 0.0
            if data['days'] > 25:
                good_attendance = 5.6
            elif data['days'] > 24:
                good_attendance = 4.2
            elif data['days'] > 20:
                good_attendance = 2.8
            elif data['days'] > 16:
                good_attendance = 1.4

            basic_amount = basic_amount + good_attendance

            to_send = {
                "advance": 0,
                "mr_advance": 0.0,
                "total_advance": 0.0,
                "attendance_amount": 0,
                "basic_amount": round(basic_amount),
                "difference_amount": 0,
                "esi": round(esi),
                "good_attendance": good_attendance,
                "grand_total": 0,
                "gross_amount": round(gross_amount),
                "incentive_amount": 0,
                "national_holiday": 0,
                "ot_amount": 0,
                "ot_hours": data['ot_hours'],
                "pf": round(pf),
                "total_ar_amount": 0,
                "empId": data["empId"],
                "salary_date": f"{year}-{f"0{month}" if month < 10 else month}-31 00:00:00.000",
                "working_days": f"{round(data["days"])}",
                "mr_days": f"{round(data["days"])}",
                "salary_month": f"{data["empId"]}_{monthName}_{year}"
            }

            # print(to_send)
            columns1 = ""
            columns2 = ""
            for key in to_send.keys():
                columns1 = columns1 + f"{key}, "
                columns2 = columns2 + f"%s, "
            columns1 = columns1[:-2]
            columns2 = columns2[:-2]

            queryString = f"INSERT INTO salary ({columns1}) VALUE ({columns2})"

            salaryCursor.execute(queryString, list(to_send.values()))
            con.commit()

            count = count + 1
    
    print(count)


con.close()

462


In [3]:
l = [1, 3, 4]

for i, e in enumerate(l):
    print(i, e)

0 1
1 3
2 4
